# Exercise with Open Weather APIs

Although most of the tasks are wrapped in functions, please feel free to test lines of code outside the function.
Once you are happy that the code is running, you can put the code in the function and test it.

In [4]:
#Import all the libraries here
import requests
import json
import pandas as pd

## Data extraction from open weather API
In this exercise, you will extract weather data about several cities and save it to a CSV file. We need the following weather information for each city:
- *city_name.*
- *country.* Name of country where city is.
- *date.* Keep only the date without time
- *hour.* Show the hour (e.g., 16, 10, 15)
- *temp.* Temperature
- *temp_min.* Minimum temperature
- *temp_max*. Maximum temperature
- *humidity.*
- *rain.* Get rain volume for the last 3 hours

Please use the description above as column names in your dataframe. For further explanattion of the output, see screenshot below.

Please retrieve and get data for the following cities: ```'Lilongwe', 'Kigali', 'Blantyre', 'Lusaka', 'Tokyo'```

The base url for the weather API is this: ```http://api.openweathermap.org/data/2.5/forecast?id=```

### Define functions to extract data from the API

In [92]:
def city_id_from_name(name= None, city_file=None):
    fopen = open(city_file, encoding="utf8")
    city_list = json.load(fopen)

    for c in city_list:
                
        if c['name'] == name:
            
           city_id = c['id']
           city_country = c['country']
           print(city_id, city_country)

    base_url = "https://api.openweathermap.org/data/2.5/"
    category = 'forecast'
    api_key = '326246a875b25fc697de7f1e5f59bbe4'
    full_url = "{}{}?id={}&APPID={}".format(base_url, category, city_id, api_key)
    response = requests.get(full_url)
    response.status_code
    json_obj = response.json()
    json_obj['list']


    data = []
    for i in json_obj['list']:
        
        date_str = i['dt_txt'][:10]
        hour_str = i['dt_txt'][11:13]  # fix: better slicing

        temp = i['main']['temp']
        temp_min = i['main']['temp_min']
        temp_max = i['main']['temp_max']
        hum = i['main']['humidity']
        rain = i.get('rain', {}).get('3h', 0)

        data_item = {
            
           'Date': date_str,
           'Hour': hour_str,
           'temp': temp,
           'temp_min': temp_min,
           'temp_max': temp_max,
           'humidity': hum,
           'rain': rain
        }

        data.append(data_item)


    df = pd.DataFrame(data)
    print(df)


city_id_from_name('Kigali',r"C:\Users\User\OneDrive\Documents\Tekher 2\city.list.json")


202061 RW
          Date Hour    temp  temp_min  temp_max  humidity  rain
0   2025-05-27   15  298.86    292.84    298.86        53  2.58
1   2025-05-27   18  295.74    289.50    295.74        67  1.93
2   2025-05-27   21  292.13    288.77    292.13        81  0.49
3   2025-05-28   00  287.82    287.82    287.82        97  0.97
4   2025-05-28   03  287.26    287.26    287.26        96  0.00
5   2025-05-28   06  291.04    291.04    291.04        90  0.12
6   2025-05-28   09  294.64    294.64    294.64        74  1.65
7   2025-05-28   12  294.64    294.64    294.64        77  1.65
8   2025-05-28   15  292.71    292.71    292.71        92  2.84
9   2025-05-28   18  288.83    288.83    288.83        95  1.31
10  2025-05-28   21  288.01    288.01    288.01        96  0.00
11  2025-05-29   00  287.13    287.13    287.13        97  0.00
12  2025-05-29   03  288.04    288.04    288.04        95  0.00
13  2025-05-29   06  289.89    289.89    289.89        93  0.00
14  2025-05-29   09  295.54   

In [81]:

def get_weather(base_url, api_key=None, city_id=None, city_name=None):
    """
    Returns current weather for this city as a dataframe
    :param base_url: the base API url
    :param api_key: your API key
    :param city_name: city name
    :param city_id: city id
    :return: DataFrame with weather info
    """

    # Add your API key to the URL and build full request URL
    url = "{}id={}&APPID={}&units=metric".format(base_url, city_id, api_key)

    # Use requests to retrieve data from the API
    response = requests.get(url)

    # Retrieve JSON from the response object
    json_obj = response.json()

    # Get city details such as name and country
    city_details = json_obj.get('city', {})
    country = city_details.get('country', '')
    city_name = city_details.get('name', '')

    # Create a list to hold the weather forecast data
    data = []

    # Loop through the forecast data
    for item in json_obj['list']:
        # Extract date text
        date_txt = item['dt_txt']

        # Handle missing rain info using get with default
        rain = item.get('rain', {}).get('3h', 0)

        # Create a dictionary for this entry
        record = {
            'city_name': city_name,
            'country': country,
            'datetime': date_txt,
            'temp': item['main']['temp'],
            'temp_min': item['main']['temp_min'],
            'temp_max': item['main']['temp_max'],
            'humidity': item['main']['humidity'],
            'rain': rain
        }

        # Add this record to the data list
        data.append(record)

    # Create a DataFrame from the data list
    df = pd.DataFrame(data)

    # Convert string datetime to datetime object
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Extract date and hour from datetime
    df['date'] = df['datetime'].apply(lambda x: x.date())
    df['hour'] = df['datetime'].apply(lambda x: x.hour)

    # Drop the original 'datetime' column if not needed
    df.drop(columns=['datetime'], inplace=True)

    # Return the final cleaned DataFrame
    print(df)

get_weather("https://api.openweathermap.org/data/2.5/forecast?","326246a875b25fc697de7f1e5f59bbe4" , 202061)




# def get_weather(base_url, api_key=None, city_id=None, city_name=None):
#     """
#     Returns current weather for this city as a dataframe
#     :param base_url --  the base API url
#     :param api_key -- your API key
#     :param city_name: city name
#     :param city_id: city id
#     :return:
#     """

#     # add your API key
#     url = "{}{}&APPID={}".format(base_url, city_id, api_key)

#     # use requests to retrieve data from the API
#     # ~ 1 line

#     # retrieve JSON from the response object above
#     # ~ 1 line

#     # inspect the json object and decide how to get the data that we need
#     # using dictionary style indexing and load the results into a list
#     # ~ 1 line

#     # We also need city information such as country, use similar
#     # dictionary indexing to retrieve the info and put into variable
#     # ~ 1 line

#     # Create a list to hold the data items, which will be daily
#     # weatheer forecasts
#     # ~ 1 line

#     # Loop through the data_items and retrieve the data we need
#     # Do the foowing in the loop
#     # 1. Create a dictionary and add 'country' and 'city_name' using city_details
#     # 2. Add the rest of the items: temp, temp_max etc to the dictionary
#     # 3. add this particullar data item to the data list
#     # in the loop, use a temporary variable to carry text date info
#     # so that you can convert it to Python date later
#     # note that in some items, rain informaion is not available
#     # find a solution to deal with this issue
#     # ~ 10-11 lines

#     # create a dataframe from the data list
#     # ~ 1 line

#     # add time aware datetime using function  pd.to_datetime()
#     # with input in the function being the date_txt column
#     # ~ 1 line


#     # add date and hour using avaiable functionality on date objects
#     # For example, to get date from datetime, do datetime.date()
#     # use df.apply(lambda x:) type of syntax for this
#     # ~ 2 lines


#     # drop columns we dont need, we only need columns specified
#     # in the problem description


#     # Remember to return the dataframe

   city_name country   temp  temp_min  temp_max  humidity  rain        date  \
0     Kigali      RW  23.18     22.51     23.18        57  1.33  2025-05-27   
1     Kigali      RW  21.87     19.26     21.87        68  2.40  2025-05-27   
2     Kigali      RW  18.71     16.48     18.71        81  0.78  2025-05-27   
3     Kigali      RW  15.23     15.23     15.23        96  0.53  2025-05-27   
4     Kigali      RW  14.61     14.61     14.61        97  0.45  2025-05-28   
5     Kigali      RW  13.88     13.88     13.88        96  0.00  2025-05-28   
6     Kigali      RW  17.72     17.72     17.72        89  0.00  2025-05-28   
7     Kigali      RW  22.53     22.53     22.53        66  0.77  2025-05-28   
8     Kigali      RW  21.67     21.67     21.67        76  1.76  2025-05-28   
9     Kigali      RW  19.58     19.58     19.58        91  3.23  2025-05-28   
10    Kigali      RW  15.62     15.62     15.62        96  2.00  2025-05-28   
11    Kigali      RW  14.81     14.81     14.81     

### Putting it all together
Define a helper functions to go through a list of countries and retrieve data by calling the functions above.

In [91]:
def retrieve_and_save_weather_data(city_list, output_csv):
    """
    Helper function putting everything together.
    Arguments:
    city_list -- list of city names to get data for
    output_csv -- full path to CSV where to save data
    """
    # Base weather API url
    base_url = "https://api.openweathermap.org/data/2.5/forecast?"

    # Use your API key here
    api_key = 'cd689df7ce5a01db2aafde528e3d87c4'

    # List to hold DataFrames for each city
    all_dataframes = []

    # Loop through list of city names
    for city_name in city_list:
        # 1. Get city ID from the JSON file
        with open(r"C:\Users\User\OneDrive\Documents\Tekher 2\city.list.json", encoding="utf8") as f:
            city_data = json.load(f)
            city_id = None
            for c in city_data:
                if c['name'].lower() == city_name.lower():
                    city_id = c['id']
                    break

        if city_id is not None:
            # 2. Get weather data for this city
            df = get_weather(base_url=base_url, api_key=api_key, city_id=city_id)
            # 3. Add the data to the list
            all_dataframes.append(df)
        else:
            print(f"City ID not found for {city_name}")

    # Combine all city data into one DataFrame and save to CSV
    final_df = pd.concat(all_dataframes, ignore_index=True)
    final_df.to_csv(output_csv, index=False)

city_list = ['Lilongwe', 'Kigali', 'Blantyre', 'Lusaka', 'Tokyo']
output_csv = 'weather_data.csv'  

retrieve_and_save_weather_data(city_list, output_csv)

   city_name country   temp  temp_min  temp_max  humidity  rain        date  \
0   Lilongwe      MW  23.01     20.80     23.01        59  0.13  2025-05-27   
1   Lilongwe      MW  20.90     19.29     20.90        65  0.24  2025-05-27   
2   Lilongwe      MW  18.45     18.45     18.45        72  0.00  2025-05-27   
3   Lilongwe      MW  16.21     16.21     16.21        83  0.00  2025-05-28   
4   Lilongwe      MW  15.82     15.82     15.82        85  0.00  2025-05-28   
5   Lilongwe      MW  18.03     18.03     18.03        74  0.00  2025-05-28   
6   Lilongwe      MW  23.53     23.53     23.53        49  0.00  2025-05-28   
7   Lilongwe      MW  25.78     25.78     25.78        41  0.00  2025-05-28   
8   Lilongwe      MW  20.06     20.06     20.06        67  0.32  2025-05-28   
9   Lilongwe      MW  18.87     18.87     18.87        71  0.17  2025-05-28   
10  Lilongwe      MW  16.87     16.87     16.87        78  0.00  2025-05-28   
11  Lilongwe      MW  15.26     15.26     15.26     

ValueError: All objects passed were None

In [ ]:
# call function here with the city names provided in the introduction.
# Make sure to also pass full-path for output CSV file
city_names = None
output_file = None